# Scope
Execute the EDA plan of action:
- Plan of action:
    - join AIS charity data with program data
    - drop records where both the parent website and program website are null (as the user can't easily contact for volunteering optys)
    - Bring through how purposes were pursued info
    - Bring through beneficiaries metadata
    - Prepare data dictionary for system prompt

In [1]:
import pandas as pd
charities = pd.read_csv("../data/datadotgov_ais22.csv")
charities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52028 entries, 0 to 52027
Data columns (total 97 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   abn                                                                            52028 non-null  int64  
 1   charity name                                                                   52028 non-null  object 
 2   registration status                                                            52028 non-null  object 
 3   charity website                                                                35299 non-null  object 
 4   charity size                                                                   52028 non-null  object 
 5   basic religious charity                                                        52028 non-null  object 
 6   ais due date          

C:\Users\geoff\AppData\Local\Temp\ipykernel_35780\2669044431.py:2: DtypeWarning: Columns (78) have mixed types. Specify dtype option on import or set low_memory=False.
  charities = pd.read_csv("../data/datadotgov_ais22.csv")


In [2]:
programs = pd.read_csv("../data/datadotgov_ais22_programs.csv")
programs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92208 entries, 0 to 92207
Data columns (total 58 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   ABN                                                               92208 non-null  int64 
 1   Charity Name                                                      92208 non-null  object
 2   Registration Status                                               92208 non-null  object
 3   Program name                                                      92195 non-null  object
 4   Classification                                                    92183 non-null  object
 5   Children - aged 6 to under 15                                     92208 non-null  object
 6   Environment                                                       92208 non-null  object
 7   Families                                

In [8]:
# first transformation script courtesy of claude
import numpy as np

def transform_charity_data(programs_df, charities_df):
    # 1. Filter for non-null websites in both dataframes
    programs_filtered = programs_df[programs_df['Charity weblink'].notna()].copy()
    charities_filtered = charities_df[charities_df['charity website'].notna()].copy()
    
    # 2. Join the dataframes on ABN/abn
    merged_df = pd.merge(
        charities_filtered,
        programs_filtered,
        left_on='abn',
        right_on='ABN',
        how='inner'
    )
    
    # 3. Create lists of location columns and their corresponding lat/long columns
    location_cols = [f'Operating Location {i}' for i in range(1, 11)]
    latlong_cols = [f'Operating Location {i} lat/long' for i in range(1, 11)]
    
    # 4. Melt the location columns
    locations_melted = pd.melt(
        merged_df,
        id_vars=[col for col in merged_df.columns if col not in location_cols + latlong_cols],
        value_vars=location_cols,
        var_name='location_number',
        value_name='operating_location'
    )
    
    # 5. Melt the lat/long columns
    latlong_melted = pd.melt(
        merged_df,
        value_vars=latlong_cols,
        var_name='latlong_number',
        value_name='lat_long'
    )
    
    # 6. Combine the melted dataframes
    final_df = locations_melted.copy()
    final_df['lat_long'] = latlong_melted['lat_long']
    
    # 7. Split lat/long into separate columns
    lat_lon_split = final_df['lat_long'].str.split('|', expand=True)
    final_df['latitude'] = pd.to_numeric(lat_lon_split[0], errors='coerce')
    final_df['longitude'] = pd.to_numeric(lat_lon_split[1], errors='coerce')
    
    # 8. Drop rows where lat/long are zero or null
    final_df = final_df.dropna(subset=['latitude', 'longitude'])
    final_df = final_df[
        (final_df['latitude'] != 0) & 
        (final_df['longitude'] != 0)
    ]
    
    # 9. Extract location number using string slicing instead of regex
    final_df['location_number'] = final_df['location_number'].str.slice(-2).str.strip().astype(int)
    
    # 10. Drop the original lat_long column as we now have split columns
    final_df = final_df.drop('lat_long', axis=1)
    
    # 11. Sort the dataframe by ABN and location number
    final_df = final_df.sort_values(['ABN', 'location_number'])
    
    return final_df

# Example usage:
data_prep = transform_charity_data(programs, charities)
data_prep.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 46738 entries, 0 to 34330
Data columns (total 139 columns):
 #    Column                                                                         Non-Null Count  Dtype  
---   ------                                                                         --------------  -----  
 0    abn                                                                            46738 non-null  int64  
 1    charity name                                                                   46738 non-null  object 
 2    registration status                                                            46738 non-null  object 
 3    charity website                                                                46738 non-null  object 
 4    charity size                                                                   46738 non-null  object 
 5    basic religious charity                                                        46738 non-null  object 
 6    ais due date     

In [9]:
# Pretty cool data, eh?
data_prep[
    [
        'abn',
        'charity name',
        'how purposes were pursued',
        'total full time equivalent staff',
        'staff - volunteers',
        'Program name',
        'Classification',
        'Charity weblink',
        'location_number',
        'operating_location',
        'latitude',
        'longitude'
    ]
].sample(10)

,abn,charity name,how purposes were pursued,total full time equivalent staff,staff - volunteers,Program name,Classification,Charity weblink,location_number,operating_location,latitude,longitude
43241,34425338462,Multicultural Youth Support Services Incorporated,Multicultural Youth Support Services (MYSS) is...,0.00,15,Homework Club,Out-of-school learning,myssvic.org.au,2,"Cranbourne West Community Hub, Flicka Boulevar...",-38.106825,145.250419
131274,85628088371,ABC Alumni Limited,We are a national organisation which provides ...,0.00,15,Community speaking and lectures,Community information,www.abcalumni.net,4,"Melbourne VIC, Australia",-37.813628,144.963058
6068,26788461065,SAMMY D FOUNDATION LIMITED,The Sammy D Foundation is a violence preventio...,9.13,52,Youth Mentoring,Youth peer mentoring,https://www.sammydfoundation.org.au/mentoring,1,"South Australia, Australia",-30.000232,136.209155
79002,38262080944,Spectrum Space Inc.,Our mission is to provide unique quality oppor...,9.94,78,YES Group,Disability services,https://www.spectrumspace.org.au/social-groups...,3,"Midland WA, Australia",-31.891857,116.013473
37176,18393396753,One in 5000 Foundation Incorporated,The ONE in 5000 Foundation provides advocacy a...,0.00,6,Rare and Resilient,Colorectal cancer,www.1in5000foundation.org.au,2,"4 Ipydene Court, Chadstone VIC, Australia",-37.878924,145.094289
197,11618913563,KARI Foundation Ltd,KARI Foundation believe culture drives change....,55.03,100,KAP AND FUNDRAISING,Economic development,www.kari.org.au,1,Australia,-25.274398,133.775136
25756,78950377819,Chabad Carnegie Community Centre Inc.,Our objective is to encourage as many Jews as ...,1.00,5,Chabad Hebrew School,Education support,chabadcarnegie.com,1,"695 North Road, Carnegie VIC, Australia",-37.905415,145.055324
48951,49780291837,ATC Sydney Inc,"Provide care, counsel, teaching normal life st...",1.00,25,Religious,Pentecostalism,http://www.tamilchurch.org.au,2,"Seven Hills NSW, Australia",-33.777917,150.931946
153248,53002941086,Haematology Society of Australia and New Zealand,HSANZ has provided scholarships and grants to ...,1.30,40,HSANZ PhD Scholarship (non-malignant haematology),Haematology,www.hsanz.org.au,5,"Queensland, Australia",-22.575197,144.084793
28467,86150627438,Inner North Playschool Inc,Provided excellent care and education for anot...,0.00,20,Inner North Playschool,Child care,http://www.innernorthplayschool.org/,1,"Ainslie ACT, Australia",-35.262164,149.145873


In [7]:
# let's pickle this for later use in prototyping
data_prep.to_pickle('../data/transformed_charities.pkl')